# Utilities

> A collection of utility functions.

In [ ]:
# | default_exp utilities

In [ ]:
# | export

from pathlib import Path
from typing import Optional

import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import torch
import torchaudio
from matplotlib.figure import Figure

from neuralresonator.dsp import apply_filter

## Plotting

In [ ]:
# | export

def db20(x: np.ndarray) -> np.ndarray:
    return 20 * np.log10(np.abs(x))

def plot_sample(
    a: np.ndarray,
    b: np.ndarray,
    gt_audio: np.ndarray,
    sample_rate: int = 16000,
    plt_ioff: bool = True,
) -> tuple[Figure, np.ndarray]:
    """Plot a sample from the model."""

    impulse = np.zeros_like(gt_audio)
    impulse[0] = 1.0

    pred_signal = apply_filter(impulse, a, b)

    pred_fft = np.fft.rfft(pred_signal, norm="forward")
    gt_fft = np.fft.rfft(gt_audio, norm="forward")

    plt.ioff() if plt_ioff else plt.ion()
    fig = plt.figure(figsize=(6, 3))
    plt.plot(db20(gt_fft), label="Ground Truth")
    plt.plot(db20(pred_fft), label="Predicted")
    plt.grid()
    plt.legend(prop={"size": 6}, loc="upper right")
    plt.tight_layout()
    plt.close(fig)

    return fig, pred_signal

Plot a dummy image

In [ ]:
# | eval: false

b = np.array([[[1.0, -1.60,  0.80]]])
a = np.array([[[1.0, -0.90,  0.27]]])

print(a.shape, b.shape)
impulse = np.zeros(int(0.3 * 16000))
impulse[0] = 1.0

gt_audio = np.random.randn(int(0.3 * 16000) + 1)

fig = plot_sample(a, b, gt_audio)
fig

(1, 1, 3) (1, 1, 3)


(<Figure size 600x300 with 1 Axes>,
 array([ 1.e+000, -7.e-001, -1.e-001, ..., -5.e-324, -5.e-324, -5.e-324]))

In [ ]:
# | export

def save_and_display_audio(
    audio: np.ndarray,
    name: str,
    sample_rate: int = 44100,
):
    "Save audio file and display in a Jupyter notebook."

    audio_dir = Path("audio")
    audio_dir.mkdir(exist_ok=True)

    out_file = audio_dir / name

    sf.write(
        file=out_file,
        samplerate=sample_rate,
        data=audio.astype(np.float32),
    )
    ipd.display(ipd.Audio(out_file))


In [ ]:
# | eval: false

save_and_display_audio(gt_audio, "gt_audio.wav")

## Losses

In [ ]:
# | export


def fft_loss(
    pred_fft: torch.Tensor,  # magnitude of the predicted fft
    target_fft: torch.Tensor,  # magnitude of the target fft
    lin_l1: float = 1.0,  # weight of the linear l1 loss
    lin_l2: float = 0.0,  # weight of the linear l2 loss
    log_l1: float = 0.0,  # weight of the log l1 loss
    log_l2: float = 0.0,  # weight of the log l2 loss
) -> torch.Tensor:  # scalar loss
    """
    Compute the loss between the target and predicted fft
    """
    diff = pred_fft - target_fft
    log_diff = torch.log(pred_fft + 1e-7) - torch.log(target_fft + 1e-7)
    return (
        diff.abs().mean() * lin_l1
        + diff.pow(2).mean() * lin_l2
        + log_diff.abs().mean() * log_l1
        + log_diff.pow(2).mean() * log_l2
    )


In [ ]:
# | export

class FFTLoss(torch.nn.Module):
    """
    Compute the loss between the target and predicted fft
    """
    def __init__(
        self,
        lin_l1: float = 1.0, # weight of the linear l1 loss
        lin_l2: float = 0.0, # weight of the linear l2 loss
        log_l1: float = 0.0, # weight of the log l1 loss
        log_l2: float = 0.0, # weight of the log l2 loss
    ):
        super().__init__()
        self.lin_l1 = lin_l1
        self.lin_l2 = lin_l2
        self.log_l1 = log_l1
        self.log_l2 = log_l2

    def forward(
        self,
        pred_fft: torch.Tensor,   # magnitude of the predicted fft
        target_fft: torch.Tensor, # magnitude of the target fft
    ):
        return fft_loss(
            pred_fft,
            target_fft,
            self.lin_l1,
            self.lin_l2,
            self.log_l1,
            self.log_l2,
        )

In [ ]:
# | export

class MelScaleLoss(torch.nn.Module):
    """
    Compute the loss between the target and predicted mel scale
    """
    def __init__(
        self,
        n_mels: int = 128,
        sample_rate: int = 16000,
        n_fft: int = 0,
        lin_l1: float = 1.0, # weight of the linear l1 loss
        lin_l2: float = 0.0, # weight of the linear l2 loss
        log_l1: float = 0.0, # weight of the log l1 loss
        log_l2: float = 0.0, # weight of the log l2 loss
        f_min: float = 0.,
        f_max: Optional[float] = 8000.,
    ) -> None:

        super().__init__()
        self.n_mels = n_mels
        self.sample_rate = sample_rate
        self.f_max = f_max if f_max is not None else float(sample_rate // 2)
        self.f_min = f_min

        self.lin_l1 = lin_l1
        self.lin_l2 = lin_l2
        self.log_l1 = log_l1
        self.log_l2 = log_l2


        assert f_min <= self.f_max, 'Require f_min: {} < f_max: {}'.format(f_min, self.f_max)

        fb = torchaudio.functional.melscale_fbanks(
            n_fft,
            n_mels=self.n_mels,
            f_min=self.f_min,
            f_max=self.f_max,
            sample_rate=self.sample_rate,
        )
        self.register_buffer('fb', fb)

    def forward(
        self,
        pred: torch.Tensor,   # magnitude of the predicted fft
        target: torch.Tensor, # magnitude of the target fft
    ) -> torch.Tensor:
        """
        Compute the loss between the target and predicted mel scale
        """
        pred_mel = torch.matmul(pred, self.fb)
        target_mel = torch.matmul(target, self.fb)
        return fft_loss(
            pred_mel,
            target_mel,
            lin_l1=self.lin_l1,
            lin_l2=self.lin_l2,
            log_l1=self.log_l1,
            log_l2=self.log_l2,
        )


## Other

In [ ]:
# | export

def to_zpk(
    x: torch.Tensor,
    n_parallel: int,
    n_biquads: int,
    gain_factor: float = 1.0,
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Convert the parameters of the filter to zeros, poles and gain

    Input:
        x: (B, n_parallel, n_biquads, 5)
    Output:
        z: (B, n_parallel, n_biquads)
        p: (B, n_parallel, n_biquads)
        k: (B, n_parallel, n_biquads)
    """

    B = x.shape[0]
    sos = x.view(B, n_parallel, n_biquads, 5)
    gain = torch.sigmoid(sos[:, :, :, 0])
    zeros = sos[:, :, :, 1:3]
    poles = sos[:, :, :, 3:5]

    p = torch.complex(poles[:, :, :, 0], poles[:, :, :, 1])
    z = torch.complex(zeros[:, :, :, 0], zeros[:, :, :, 1])
    k = gain_factor * gain
    return z, p, k